# `mult` processor demo

In [ ]:
USE_PRIVATE_DISTRO = True

%tensorflow_version 2.x
if USE_PRIVATE_DISTRO:
    print("[INFO] Using private distro. Be careful.")
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -qU /content/drive/MyDrive/SMC\ 10/DDSP-10/dist/ddsp-1.2.0.tar.gz
else:
    !pip install -qU ddsp

# Ignore a bunch of deprecation warnings
import warnings
warnings.filterwarnings("ignore")

import ddsp
import ddsp.training
from ddsp.colab.colab_utils import play, specplot
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
sample_rate = 48000 

n_frames = 1000
hop_size = 64
n_samples = n_frames * hop_size

In [ ]:
note = ddsp.synths.Harmonic(n_samples=n_samples, 
                            sample_rate=sample_rate, 
                            name='note')
lfo = ddsp.synths.Harmonic(n_samples=n_samples, 
                            sample_rate=sample_rate, 
                            name='lfo')
mult = ddsp.processors.Mult(name='mult')

dag = [
  (note, ['gain', 'harmonic_amps', 'f0_hz']),
  (lfo, ['lfo_gain', 'lfo_harmonic_amps', 'lfo_f0_hz']),
  (mult, ['note/signal', 'lfo/signal']),
]

In [ ]:
n_harmonics = 1

# Create a sinusoid
gain = 0.2*np.ones(n_frames)[np.newaxis, :, np.newaxis]
harmonic_amps = np.ones((n_frames, n_harmonics))[np.newaxis, :, :]
f0_hz = 440.0 * np.ones(n_frames)[np.newaxis, :, np.newaxis]

# Create an LFO
lfo_gain = np.ones(n_frames)[np.newaxis, : , np.newaxis]
lfo_harmonic_amps = np.ones((n_frames,n_harmonics))[np.newaxis, :, :]
lfo_f0_hz = 3.0 * np.ones(n_frames)[np.newaxis, :, np.newaxis]

inputs = {
    'gain': gain,
    'harmonic_amps': harmonic_amps,
    'f0_hz': f0_hz,
    'lfo_gain': lfo_gain,
    'lfo_harmonic_amps': lfo_harmonic_amps,
    'lfo_f0_hz': lfo_f0_hz,
}
inputs = {k: v.astype(np.float32) for k, v in inputs.items()}

processor_group = ddsp.processors.ProcessorGroup(dag=dag)
audio_out = processor_group(inputs)

play(audio_out, sample_rate=sample_rate)
specplot(audio_out)